In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

Using TensorFlow backend.


In [3]:
train_df = pd.read_csv("mall/m_623.csv", header = 0)

FileNotFoundError: File b'mall/m_623.csv' does not exist

In [ ]:
train_df = train_df.drop(train_df.columns[0], axis = 1)

In [ ]:
#train_AP_strengths = train_df.ix[:,:4070]
train_AP_strengths = train_df.drop("shop_id", axis = 1)
train_AP_features = scale(np.asarray(train_AP_strengths))

In [ ]:
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train_df['shop_id'].values))
train_labels = lbl.transform(list(train_df['shop_id'].values))
dummy_labels = pd.get_dummies(train_labels)
dummy_labels.shape

In [ ]:
train_labels = np.asarray(dummy_labels)

In [ ]:
train_X = train_AP_features
train_y = train_labels

train_val_split = np.random.rand(len(train_AP_features))
train_val_split = train_val_split < 0.70
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X  = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [ ]:
nb_epochs = 100
batch_size = 256
input_size = train_X.shape[1]
num_classes = train_y.shape[1]

In [ ]:
def encoder():
    model = Sequential()
    model.add(Dense(256, activation = "relu", input_dim = input_size))
    model.add(Dense(128, activation = "relu"))
    model.add(Dense(64, activation = "relu"))
    return model

In [ ]:
def decoder(e):
    e.add(Dense(128, activation = "relu", input_dim = 64))
    e.add(Dense(256, activation = "relu"))
    e.add(Dense(input_size, activation = "relu"))
    e.compile(optimizer = "adam", loss = "mse")
    return e

In [ ]:
e = encoder()
d = decoder(e)

In [ ]:
d.fit(train_X, train_X, epochs = nb_epochs, batch_size = batch_size, verbose = 0)

In [ ]:
sgd = SGD(lr = 0.01, decay=1e-6, momentum = 0.9, nesterov = True)
def classifier(d):
    d.add(Dense(128, activation = "relu",input_dim = 64))
    d.add(Dropout(0.5))
    d.add(Dense(128, activation = "relu"))
    d.add(Dropout(0.5))
    d.add(Dense(num_classes, activation = "softmax"))
    d.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
    return d

In [ ]:
c = classifier(d)

In [ ]:
c.fit(train_X, train_y, validation_plit = 0.1, epochs = nb_epochs, batch_size = batch_size)

In [ ]:
test_X = pd.read_csv("m_4495_test.csv")

In [ ]:
test_X = test_X.drop(test_X.columns[0], axis = 1)

In [ ]:
test_AP_features = scale(np.asarray(test_X))

In [ ]:
results = np.argmax(results , axis = 1)

In [ ]:
results = pd.Series(results, name = "label")

In [ ]:
results

In [ ]:
results.head()

In [ ]:
test_results = lbl.inverse_transform(results)

In [ ]:
test_results

In [ ]:
test_X['shop_id'] = test_results

In [ ]:
test_X.head()

In [ ]:
submission = pd.read_csv("input/evaluation_public.csv")

In [ ]:
submission_m_4495 = submission[submission.mall_id == "m_4495"]
submission_m_4495['shop_id'] = test_results

In [ ]:
submission_m_4495.head()

In [ ]:
r = submission_m_4495[['row_id', 'shop_id']]

In [ ]:
r.head()

In [ ]:
r.to_csv('sub.csv', index = False)